# Feature Pyramid Networks for Object Detection

- __ 발표자 : 정지원__
- __ 발표일 : 2018. 01. 14__
---

## Abstarct

Feature pyramids는 서로 다른 스케일의 오브젝트 디텍션에 사용되는 기본적인 요소이다. 그러나 최신 딥러닝 오브젝트 디텍터는 피라미드 구조를 따르지 않는데, 계산과 메모리가 많이 들기 때문이기도 하다. 이 논문에서는 marginal extra cost가 들어가는 Feature Pyramid를 구성하기 위해 깊은 Conv-networks의 멀티 스케일 피라미드 계층 구조를 이용한다. 옆 방향으로의 연결을 갖는 top-down 아키텍처는 모든 scale에서 상위 수준의 의미론적 feature map을 작성하기 위해 개발된다. 이런 아키텍처를 Feature Pyramid Network(FPN)이라 부르며, 여러 응용분야에서 사용될 일반적인 feature 추출기로서의 상당한 개선을 보여줬다. 기본적인 Faster R-CNN에서 FPN을 사용하는 것은 COCO detection 데이터에 대해 SOTA 성능을 보였다. 또한, GPU에서 6FPS를 보였기 때문에, 일반적인 멀티스케일 Object detection에 대한 실용적이고 좋은 솔루션이라 할 수 있다.

## 1. Introdction

![Image](figures/1.png)

다양한 스케일의 객체를 detect하는 문제는 컴퓨터 비전 분야에서 기본적인 챌린지였다. 위 그림에서의 (A)Image pyramid를 사용하는 방식은 기본적인 솔루션이었다. 이러한 피라미드는 피라미드의 레벨을 이동시킴으로써 객체의 스케일 변경이 상쇄된다는 점에서 scale-invariant다. 직관적으로, 이 방식을 사용하면 모델에서 위치와 피라미드 레벨 모두에서 모델을 스캔하여 광범위한 범위의 물체를 감지할 수 있다.

Featurized image pyramids는 hand-engineered feature를 사용하던 시절에는 많이 사용됐었다. 좋은 결과를 얻게 위해서는 이러한 방식이 필수였기 때문이다. Recognition 문제에서, engineered feature는 deep-convolutional networks로 많이 대체됐다. 더 높은 수준의 의미를 표현할 수 있을뿐만 아니라, ConvNets은 scale의 variance에 보다 robust하기 때문에 단일 입력 스케일에서 계산된 피처로부터의 recognition을 가능하게 했다. 그러나 이러한 강건함에도 불구하고, pyramids는 여전히 더 정확한 결과를 필요로 했다. Imagenet이나 COCO detection challenge에서의 최근 top entries는 featurized image pyramids에서 multi-scale testing을 사용한 방법이었다. Image pyramid의 각 레벨을 featurizing하는 것의 주된 이점은 고해상도 레벨을 포함하여 모든 레벨이 의미론적으로 강한 multi-scale feature representation을 생성한다는 것이다.

그럼에도 불구하고 이미지 피라미드의 각 레벨을 featurize하는 것은 분명히 한계가 있다. Inference 시간은 상당히 증가하여, 이 접근법은 실제 응용 프로그램에서는 비실용적이다. 더욱이 end-to-end로 모델을 학습시키는 것은 메모리상으로 불가능하며, 만약 사용하더라도 테스트시에만 사용하게 되어, train/test inference time에 차이를 만든다. 이러한 이유로, Fast와 Faster R-CNN은 featurized image pyramid를 기본 설정으로 사용하지 않았다.

그러나, image pyramids가 multi-scale feature representation을 구하는 유일한 방법은 아니다. Deep ConvNet은 feature hierarchy를 layer마다 구해내며, subsampling layers를 사용하여 feature hierarchy는 고유한 multi-scale를 갖게 된다. 이러한 in-network feature hierarchy는 feature map을 다른 spatial resolutions으로 만들어내지만, 서로 다른 depths로 인해 큰 semantic gap을 발생시킨다. 이 고해상도 map은 low-level features를 갖는데, 이런 것들이 object recognition을 위한 representational capacity에 좋지 못하다.

Single Shot Detector(SSD)는 ConvNet의 pyramidal feature hierarchy를 사용하는 것을 처음으로 시도한 방식이었다. 위 그림에서의 C에 해당한다. 이상적으로, SSD-style pyramid는 multi-scale feature map을 이미 다 계산된 서로 다른 layer에서 가져다 쓰므로 계산비용이 들지 않는다. SSD는 이미 계산된 레이어를 재사용하지 않고 대신 네트워크의 최고점부터 시작하여 몇 가지 새로운 레이어를 추가하여 피라미드를 만든다. 
따라서 feature hierarchy의 고해상도 map을 다시 사용할 수 있는 기회를 놓치게 된다. 우리는 이것이 작은 물체를 탐지하는데 중요하다는 것을 확인했다.

이 논문의 목표는 자연스럽게 ConvNet의 피쳐 계층 구조의 피라미드 모양을 활용하고 모든 피라미드에서 강력한 의미를 갖도록 하는 것이다. 이 목표를 달성하기 위해 우리는 top-down와 lateral connections을 통해 의미론적으로 강력한 feature를 가진 저해상도와 의미상 약한 feature를 가진 고해상도를 결합한 아키텍처를 사용한다. 이는 모든 레벨에서 풍부한 의미를 가지며 단일 입력 이미지 스케일에서 신속하게 만들어지는 피라미드다. 표현력, 속도 또는 메모리를 희생하지 않고 featurized된 이미지 피라미드를 대체하는데 사용할 수있는 in-network 피라미드를 만드는 방법을 보여준다.

![Image](figures/2.png)

최근의 연구에서 top-down과 skip connection을 채택한 비슷한 아키텍처가 인기있다. 그들의 목표는 예측을 할 좋은 해상도의 single high-level feature map을 생성하는 것이다.(그림 중 위) 반대로, 논문에서 사용한 방법은 예측을 각 레벨에서 독립적으로 이루어지게 하는 피라미드 아키텍처를 이용한다.(그림 중 아래) 우리 모델은 featurized 된 이미지 피라미드를 echo한다.

FPN이라 불리는 이 방식에 대해 detection과 segmentation에 대해 평가를 했다. 단순히 FPN방식을 Faster R-CNN에 적용하여 COCO detection benchmark에 대해 SOTA 성능을 보였다. 절제된 실험에서, bouding box proposal에 대해 FPN은 Average Recall을 8.0 point까지 올렸다. Faster R-CNN on Resnet을 베이스라인으로 하여, COCO에서는 2.3points, PASCAL에서는 3.8points를 향상시켰다. 우리의 방식은 mask proposal 등에도 쉽게 적용될 수 있으며, instance segmentation AR과 SOTA 방식의 속도를 향상시켰다.

게다가, 우리의 피라미드 구조는 end-to-end로 모든 스케일에 대해 학습되며, train/test time도 일관적이다. 결과적으로 FPN은 존재하는 모든 SOTA 방법보다 더 좋은 정확도를 얻을 수 있었다. 이러한 성과는 testing time의 증가 없이 이루어졌다. 이러한 발전이 미래의 연구나 적용에 많은 도움이 될 것이라 믿는다.

## 2. Related Work

#### Hand-engineered features and early neural networks.

SIFT features were originally extracted at scale-space extrema and used for feature point matching. HOG features, and later SIFT features as well, were computed densely over entire image pyramids. These HOG and SIFT pyramids have been used in numerous works for image classification, object detection, human pose estimation, and more. There has also been significant interest in computing featurized image pyramids quickly. Dolla ́retal. demonstrated fast pyramid computation by first computing a sparsely sampled (in scale) pyramid and then interpolating missing levels. Before HOG and SIFT, early work on face detection with ConvNets computed shallow networks over image pyramids to detect faces across scales.

#### Deep ConvNet object detectors.

With the development of modern deep ConvNets, object detectors like OverFeat and R-CNN showed dramatic improvements in accuracy. OverFeat adopted a strategy similar to early neural network face detectors by applying a ConvNet as a sliding window detector on an image pyramid. R-CNN adopted a region proposal-based strategy in which each proposal was scale-normalized before classifying with a ConvNet. SPPnet demonstrated that such region-based detectors could be applied much more efficiently on feature maps extracted on a single image scale. Recent and more accurate detection methods like Fast R-CNN and Faster R-CNN advocate using features computed from a single scale, because it offers a good trade-off between accuracy and speed. Multi-scale detection, however, still performs better, especially for small objects.

#### Methods using multiple layers.

A number of recent approaches improve detection and segmentation by using different layers in a ConvNet. FCN sums partial scores for each category over multiple scales to compute semantic segmentations. Hypercolumns uses a similar method for object instance segmentation. Several other approaches (HyperNet, ParseNet, and ION) concatenate features of multiple layers before computing predictions, which is equivalent to summing transformed features. SSD and MS-CNN predict objects at multiple layers of the feature hierarchy without combining features or scores.

There are recent methods exploiting lateral/skip connections that associate low-level feature maps across resolutions and semantic levels, including U-Net and SharpMask for segmentation, Recombinator networks for face detection, and Stacked Hourglass networks for keypoint estimation. Ghiasi et al. present a Laplacian pyramid presentation for FCNs to progressively refine segmentation. Although these methods adopt architectures with pyramidal shapes, they are unlike featurized image pyramids where predictions are made independently at all levels, see Fig. 2. In fact, for the pyramidal architecture in Fig. 2 (top), image pyramids are still needed to recognize objects across multiple scales.

## 3. Feature Pyramid Networks

우리의 목표는 ConvNet의 pyramidal feature hierarchy 구조를 활용하는 것이다. 이 pyramid feature 계층 구조는 낮은 수준에서 높은 수준까지의 semantics을 가지며 전체적으로 높은 수준의 의미를 갖는 피라미드를 만든다. 피처 피라미드 네트워크의 결과는 일반적인 목적이며, 이 문서에서는 슬라이딩 윈도우 proposer(RPN) 및 region-based detector(Fast R-CNN)에 중점을 둔다. 

우리의 방식은 Fully-convolutional 구조를 통해, single-scale image를 임의의 사이즈로 받아서 여러 levels에서의 feature maps의 사이즈와 비례하여 outputs을 낸다. 이러한 과정은 backbone Convolutional 아키텍처와는 독립적이며, 이 논문에서는 ResNet을 사용한 결과를 보여준다. 우리의 피라미드 구축은 bottom-up pathway, top-down pathway, lateral connections을 포함한다.

#### Bottom-up pathway.

Bottom-up pathway는 Backbone Convnet의 feedworward 계산이며, 2단계의 scaling을 통해 다양한 scale의 feature map으로 구성된 feature hierarchy를 계산해낸다. 동일한 크기의 output map을 생성하는 layer가 많으며, 이는 즉 이러한 layers들이 같은 network stage에 있음을 의미한다. 우리의 feature pyramid에서는, 각 stage마다 하나의 pyramid를 정의한다. 각 stage에서 last layer의 output을 Feature pyramid maps의 대표로서 선택한다. 이를 통해, 우리의 피라미드를 잘 만들도록 하게 된다. 각 단계에서의 가장 깊은 layer가 가장 강력한 features를 갖기 때문에, 이는 자연스러운 선택이다.

특히 ResNet은 각 stage의 마지막 residual block에서 출력되는 feature activation를 사용하게 된다. 큰 메모리 공간으로 인해, 첫 번째 conv-layer는 pyramid에 포함하지 않는다.

#### Top-down pathway and lateral connections.

top-down pathway는 고차원 피라미드 레벨로 feature maps을 Upsampling을 하면서 hallucinate하지만 의미상(semantic)으로는 강하게 만든다. 이러한 features는 lateral connections을 통해 bottom-up pathway의 feature로 향상된다. 각각의 lateral connections은 bottom-up pathway와 top-down pathway의 동일한 공간 사이즈의 feature map들을 병합한다. bottom-up feature map은 낮은 레벨의 semantics이지만 이것의 activations은 더욱 localized된 정보를 가지고 있게 된다.

![Image](figures/3.png)

위 그림은 top-down feature map을 구성하는 building block을 보여준다. 더 거친 해상도의 feature map을 사용하여 spatial resolution을 2배 위로 Upsampling한다.(간략화를 위해 가장 가까운 이웃 업 샘플링 사용). Upsampling된 map은 상응하는 bottom-up map(채널을 줄이기 위해 1×1 conv를 거친)과 elementwise addition하여 병합된다. 이 과정은 최상의 resolution map이 생성될 때까지 반복된다. 반복을 시작하기 위해 C5에 1x1 conv-layer를 더하여 가장 조밀한 해상도 맵을 만든다. 마지막으로 각 병합된 맵에 3x3 conv을 추가하여 upsampling의 aliasing 효과를 줄이는 최종 feature map을 생성한다. 이 최종 set의 feature map은 각각 {P2, P3, P4, P5}로 불리며, 각각 동일한 공간 크기인 {C2, C3, C4, C5}에 대응한다.

피라미드의 모든 레벨은 기존 featurized 된 Image pyramid에서와 같이 공유되는 classifier/regressors를 사용하기 때문에 모든 feature map에서 feature dimension (채널 수, d로 표시)를 고정한다. 이 논문에서는 d = 256으로 설정했으므로 여분의 모든 Conv layer에는 256채널 출력이 있다. 이러한 여분의 layer에는 비선형성이 없으며, 경험적으로 이것이 영향이 거의 없음을 발견했다.

단순성이 우리 design의 핵심이며, 모델이 많은 design에서 강건함을 발견했다. 우리는 보다 정교한 블록(예:using multi-layer residual blocks as the connections)을 실험했으며 약간 더 나은 결과를 확인했다. 더 나은 connection module 설계는 이 백서의 초점이 아니므로 위에서 설명한 간단한 설계를 선택했다.

## 4. Applications

우리의 방법은 깊은 ConvNets 내부에 피라미드를 구축하기 위한 일반적인 솔루션이다. 다음에서는 bounding box proposal 생성을 위해 RPN에서, 그리고 Object detection을 위해 Fast R-CNN에서 우리의 방법을 채택한다. 우리 방법의 단순성과 효과성을 입증하기 위해 우리는 feature pyramid에 적용 할 때, 원래 시스템을 최소한으로 수정했다.

### 4.1 Feature Pyramid Networks for RPN

RPN은 sliding-window 클래스에 독립적인 Object detector이다. 원래의 RPN 디자인에서 작은 subnetwork는 조밀한 3x3 슬라이딩 윈도우에서 단일 스케일 conv feature map에 평가되어 객체/비객체 binary 분류 및 bounding box regressor를 수행한다. 이것은 3x3 Conv layer와 Classfication 및 regression을 위한 2개의 sibling 1x1 conv에 의해 실현되며 network head라고 부른다. 객체/비객체 기준과 bounding box regressor 목표는 앵커라고 불리는 set of reference로 정의된다. 앵커는 모양이 다른 객체를 커버하기 위해 미리 정의된 여러 가지 scale과 aspect로 되어 있다.

Single-scale feature map을 FPN으로 대체하여 RPN을 적용한다. 피라미드의 각 레벨에는 동일한 디자인(3x3 conv 및 2개의 sibling 1x1 conv) 헤드를 부착한다. 헤드가 모든 피라미드 레벨의 모든 위치에 밀집되어 있기 때문에 특정 레벨에 멀티 스케일 앵커가 있을 필요는 없다. 대신 각 척도에 단일 척도의 앵커를 지정한다. 공식적으로 우리는 앵커가 {P2, P3, P4, P5, P6}에 $\{32^{2}, 64^{2}, 128^{2}, 256^{2}, 512^{2}\}$ 픽셀의 영역을 갖도록 정의한다. RPN 논문에서와 같이 여러 aspect ratios의 앵커 각 수준에서 {1:2, 1:1, 2:1}를 사용한다. 따라서 전체적으로 피라미드 위에 15개의 앵커가 있다.

우리는 RPN에서와 같이 GT BB와의 IoU(Intersection-Over-Union)비율에 따라 앵커에 training label을 지정한다. 형식적으로 앵커는 주어진 GT BB에 대해 가장 높은 IoU를 갖거나 모든 GT BB에 대해 0.7을 초과하는 IoU를 가지면 [+] label이 할당되고, GT BB에 대해 IoU가 0.3 미만인 경우 [-] 레이블이 지정된다. GT BB의 scale은 피라미드의 수준에 지정하기 위해 명시적으로 사용되지 않는다. 대신, GT BB는 피라미드 레벨에 할당된 앵커와 연결된다. 이와 같이 RPN의 규칙 외에 추가 규칙은 없다.

헤드의 파라미터는 모든 피라미드 레벨에서 공유된다. 우리는 파라미터를 공유하지 않고 대안을 평가했으며 비슷한 정확성을 관찰했다. 파라미터를 공유하면 피라미드의 모든 수준이 비슷한 의미론적 수준을 공유한다는 것을 알 수 있다. 이 장점은 임의의 이미지 스케일에서 계산된 피쳐에 공통 헤드 분류기를 적용할 수 있는 featurized된 이미지 피라미드를 사용하는 것과 유사하다.

위의 적용으로, RPN은 FPN에서 자연스럽게 훈련되고 테스트 될 수 있다. 우리는 실험에서 구현 세부 사항을 자세히 설명한다.

### 4.2. Feature Pyramid Networks for Fast R-CNN

Fast R-CNN은 Region-of-Interest(RoI) pooling feature을 추출하는데 사용되는 region-based Object detector이다. Fast R-CNN은 가장 일반적으로 single-scale feature map서 수행된다. FPN과 함께 사용하려면 피라미드 레벨에 다른 스케일의 RoI를 할당해야 한다.

피라미드는 이미지 피라미드에서 만들어진 것처럼 보인다. 따라서 우리는 이미지 기반 피라미드에서 실행되는 경우 region-based detector의 할당 전략을 적용 할 수 있다. 공식적으로 우리는 너비 w와 높이 h의 입력 영상을 우리 피쳐 피라미드의 레벨 Pk에 할당해야 한다.

### $k= \lfloor k_{0} + log_{2}(\sqrt{wh} / 224) \rfloor$

여기서 224는 표준 Imagenet 사전 학습 크기이며, $k_0$은 $ w * h = 224^{2}$를 갖는 RoI가 mapping되어야 하는 target level이다. ResNet based Faster R-CNN 시스템과 유사하게 $C_{4}$를 single-scale feature map으로 사용하였고, $k_{0}$은 4로 설정하였다. 위 식은 RoI의 scale이 작아지면,(224의 1/2배) 정밀한 수준으로 매핑되어야 함을 의미한다.

우리는 모든 레벨의 모든 RoI에게 predictor head를 부착한다. (Fast R-CNN에서 헤드는 클래스 별 Classifier/Regressor를 의미함). 다시 말하지만, head는 모두 레벨에 관계없이 파라미터를 공유합니다. ResNet의 conv5 layer(9-layer deep subnetwork)가 conv4 feature 상단에 채택되었지만, 우리 방식에서는 이미 conv5를 사용하여 피라미드를 구성했다. 따라서 ResNet과 달리, 우리는 단순히 RoI 풀링을 채택하여 7×7 feature를 추출하고 최종 Classifier 및 BB Regressor 레이어 앞에 숨겨진 1024차원의 Fully-connected layer (각각 ReLU가 뒤따라야 함)를 붙인다. 이 layer는 무작위로 초기화되며, ResNets에서 사전에 훈련된 FC-layer가 없기 때문에 무작위로 초기화된다. 표준 conv5 head에 비해 2-fc MLP 헤드는 더 가볍고 빠르다.

## 5. Experiments on Object Detection

우리는 80 종류의 COCO 검출 데이터 세트에 대한 실험을 수행한다. 우리는 80k 학습 이미지와 35k subset 이미지의 조합을 사용하여 학습하고, 5k 이미지의 로 평가한다. 우리는 또한 공개된 레이블이 없는 표준 테스트 세트(test-std)에 대한 최종 결과를 보고한다.

일반적인 관행에서와 같이, 모든 네트워크 백본은 ImageNet-1k 분류 세트에서 사전 훈련된 다음 탐지 데이터 세트에서 fine-tuning된다. 우리는 공개적으로 사용 가능한 미리 훈련된 ResNet-50 및 ResNet-101 모델을 사용한다. 우리 코드는 Caffe2를 사용하는 py-faster-rcnn의 재구현이다.

### 5.1. Region Proposal with RPN

COCO-style Average Recall을 평가했으며, small/medium/large object에 대해 평가된다. 이미지 당 100개 또는 1000개의 proposals에 대한 결과를 보고한다.

![Image](figures/4.png)

#### Implementation details

위 테이블에 있는 모든 아키텍처는 end-to-end로 학습이 됐다. 입력 이미지의 작은 쪽 길이가 800pixel이 되도록 resize했다. 8개의 GPU로 Synchronized SGD training을 수행했다. GPU당 mini-batch에 2개의 이미지, 각 이미지당 256개의 앵커가 존재한다. decay=0.0001, momentum=0.9였다. lr=0.02 for 30k, 0.002 for next 10k. baseline을 포함한 모든 RPN 실험에서는 학습동안 image를 벗어난 BB도 포함시켰다. 구체적인 구현사항은 Faster-RCNN 논문을 참고하면 된다. 8개의 GPU로 COCO를 학습시키는데 약 8시간이 걸렸다.

#### 5.1.1 Ablation Experiments

#### Comparisons with baselines

Original RPN과의 공정한 비교를 위해, 두개의 베이스라인은 single-scale map을 사용하고, 같은 hyper-parameter를 사용했다.

FPN을 RPN에 적용한 결과, 테이블과 같이 많이 향상된 것을 알 수 있다.

#### How important is top-down enrichment?

위 표의 (d)는 top-down 경로가 없는 피라미드의 결과를 보여준다. 이 수정으로 3x3 conv를 뒤따르는 1x1 lateral connections이 bottom-up 피라미드에 부착된다. 이 아키텍처는 피라미드 피처 계층을 재사용하는 효과를 시뮬레이션한다.

(d)의 결과는 RPN baseline과 동등하며 우리 뒤에서 지연된다. 우리는 이것이 bottom-up 피라미드에서 서로 다른 레벨 사이에 큰 semantic 상의 차이(특히, 매우 깊은 ResNets의 경우)가 존재하기 때문이라고 추측한다. 우리는 또한 head의 매개 변수를 공유하지 않고 (d)의 변형을 평가했지만, 비슷한 성능 저하를 관찰했다. 이 문제는 레벨 별 head로 간단히 해결할 수 없다.

#### How important are lateral connections?

1x1 lateral connections이 없을 때의 차이를 테이블1(e)에서 보여준다. 이러한 top-down pyramid는 strong semantic feature와 fine resolutions을 갖는다. 그러나 우리는 이러한 feature map들이 여러 번 downsampled되고 upsampled되면서 feature의 location이 정확하지 않을 것이라 생각했다. 더욱 정확한 feature의 location은 bottom-up map의 finer한 level로부터 lateral connections을 통해 직접적으로 전달된다. 결과적으로 FPN은 $AR^{1k}$에서 10 points 더 좋은 결과를 보였다.

#### How important are pyramid representations?

피라미드 representations에 의지하는 대신, 가장 높은 해상도

Instead of resorting to pyramid representations, one can attach the head to the highest-resolution, strongly semantic feature maps of P2 (i.e., the finest level in our pyramids). Similar to the single-scale baselines, we assign all anchors to the P2 feature map. This variant (Table 1(f)) is better than the baseline but inferior to our approach. RPN is a sliding window detector with a fixed window size, so scanning over pyramid levels can increase its robustness to scale variance.
In addition, we note that using P2 alone leads to more anchors (750k, Table 1(f)) caused by its large spatial resolution. This result suggests that a larger number of anchors is not sufficient in itself to improve accuracy.

### 5.2. Object Detection with Fast/Faster R-CNN

![Image](figures/5.png)
![Image](figures/6.png)

이제 region-based detector에 쓰인 FPN에 대해 알아본다. object detection은 COCO-style Average Precision과 PASCAL-stype AP로 평가했다. 또한 small/medium/large objects에 대한 각각의 COCO AP도 보고한다.

#### Implementation details

input image는 길이가 최소 800 pixels 이상이 되도록 resize했다. Synchronized SGD는 GPU 8개로 학습이 됐다. 각 Mini-batch는 GPU마다 2개의 이미지씩, 그리고 이미지당 512개의 RoI를 처리하였다. decay=0.0001, momentum=0.9를 사용했다. lr=0.02 for 60k, 0.002 for 20k를 썼다. Training에서 이미지당 2000개의 RoI를 사용하였으며, testing에서는 1000개를 사용했다. FPN을 적용한 Fast R-CNN은 COCO데이터에 대해 10시간이 걸렸다.

#### 5.2.1 Fast R-CNN (on fixed proposals)

Region-based detector에서의 FPN의 효과를 더 잘 입증하기 위해서 _fixed set of proposals_에 대해서 Fast R-CNN을 ablation을 수행했다. FPN을 통한 RPN에서 계산된 proposals로 고정시키는데, 작은 object 검출에 좋은 성능을 보이기 때문이다. 간단하게 하기 위하여, Fast R-CNN과 RPN간의 feature를 공유하지 않았다.

ResNet-based Fast R-CNN baseline으로, RoI pooling으로 output-size가 14x14가 되도록 맞췄다. 테이블 2에서와 같은 성능을 보였다.

테이블2(c)는 Fast R-CNN에 FPN을 적용한 우리의 결과를 보여준다. 베이스라인들보다 좋은 성능을 보임을 볼 수 있다.

또한 제한된 상황에서의 실험을 통해, later connections을 제거한 네트워크에서의 결과와, top-down방식을 제거한 네트워크에서의 결과도 볼 수 있다. 

![Image](figures/7.png)

#### 5.2.2 Faster R-CNN (on consistent proposals)

위에서는 fixed set of proposals을 사용했다. 그러나 Faster R-CNN에서, Fast R-CNN과 RPN은 feature sharing을 하기 위해 정확히 같은 back-bone을 이용해야한다. 테이블 3은 우리의 방식과 두개의 base-line과의 차이를 보여준다.

- (i) 600이 아닌 800 pixels scale을 사용
- (ii) 64 RoIs가 아닌 512 RoIs per Image를 사용
- (iii) 4개가 아닌 5개 scale anchors 사용
- (iv) 테스트 시에 300개가 아닌 1000개의 proposals 사용

이런 차이점으로 인해 ResNet-50 Faster R-CNN보다 더 좋은 결과를 얻어냈다.

#### Sharing features

단순함을 위하여, RPN과 Fast R-CNN의 features를 공유하지 않았다. 실험을 통해, Sharing features를 하는 것이 좀 더 좋은 결과를 내는 것을 알 수 있었다. 또한 Testing time을 줄여주는 역할도 했다.

#### Running time

Feature Sharing을 함으로써, FPN-based Faster R-CNN ResNet-50에 대해 NVIDIA M40 GPU를 사용하여 inference time을 0.148s가 나왔다.(ResNet-101은 0.172s) 일반적인 single-scale ResNet-50 baseline의 경우 0.32s가 걸린다. FPN에서 extra layers때문에 적은 extra cost가 발생한다고 하였으나, 실제로 lighter한 weights를 갖는 head를 갖는다. 결국 모델 자체는 더 빠르게 작동한다. 이러한 효율성과 간결함이 추후 연구나 적용에서 좋게 쓰이길 바란다.

![Image](figures/8.png)

#### 5.2.3 Comparing with COCO Competition Winners

위 테이블에서, 우리의 ResNet-101 모델은 default learning rate schedule 때문에 충분히 학습이 되지 않은 것을 확인하였다. 따라서, 우리는 각 learning rate에 대해 Fast R-CNN step이 학습될 때마다 Mini-batches의 수를 2배로 증가시켰다. 이로인해 sharing features 없이도 AP를 증가시킬 수 있었다. 이 모델로 COCO detection leaderboard에서 SOTA의 성능을 보였다. 제한된 시간으로 인해 feature-sharing 버전을 제출하진 못했지만, 살짝 더 좋은 성능을 보이기도 했다.

더욱이, 우리의 방법은 아직 여러 많은 개선에 적용해보지 못했다. iterative regression, hard negative mining, context modeling, stronger data augmentation 등등... FPN을 적용하여 더 좋은 결과를 얻어낼 수 있을 것이다.

최근에, FPN은 COCO competetion에 대해 all tracks에서 top results를 냈다.(detection/segmentation/keypoint estimation)

## 6. Extensions: Segmentation Proposals

![Image](figures/9.png)

FPN을 통해 segmentation proposals을 도출하는데도 사용하였다. DeepMask/SharpMask는 이미지의 instance segments를 찾아내기 위해 학습된다. Inference단계에서 모델은 이미지의 dense proposals을 만들기 위해 convolutionally 계산된다. 다양한 scales에서의 segments를 생성하기 위해서, 이미지 피라미드는 필수이다.


FPN을 mask proposals에 적용하는 것은 쉽다. Training과 inference 둘 다 Fully convolutional setup을 하였다. 우리는 feature pyramid를 RPN구조(Sec 5.1)로 만들고 d=128로 두었다. 각 feature pyramid 레벨의 top에서, 14x14 mask와 object scores를 예측하기 위해 5x5의 MLP 구조를 적용하였다. 자세한 내용은 Appendix에서 다룬다.

![Image](figures/10.png)

### 6.1. Segmentation Proposal Results

여러 시도를 통해 위 결과를 얻었다. 5x5 MLP/7x7 MLP/output 14x14/output 28x28

DeepMask/SharpMask/InstanceFCN 등 mask proposal generation에 대해서도 SOTA 성능을 얻었다. 특히나 small objects에 대한 성능은 거의 2배를 기록했다. 기존의 mask proposal methods의 경우 계산이 매우 많이 들어갔지만, FPN을 사용하면 더 빠르게 작동이 가능하다. 이 결과는 우리 모델이 일반적인 feature extractor이고, 다른 multi-scale detection 문제에 대한 이미지 피라미드를 대체할 수 있음을 보여준다.

## 7. Conclusion

우리는 ConvNets 내부에 피라미드를 만들기 위해 깔끔하고 간단한 프레임 워크를 제시했다. 우리의 방법은 몇 가지 강력한 base-line 및 Competition 우승자보다 중요한 improvements를 보여준다. 따라서 image pyramids를 계산할 필요없이 feature pyramid의 연구 및 응용을 위한 실용적인 솔루션을 제공한다. 마지막으로 우리의 연구는 Deep ConvNets의 강력한 representational power와 scale variation에 대한 implicit robustness에도 불구하고, 피라미드 표현을 사용하여 multi-scale 문제를 명시적으로 처리하는 것은 여전히 중요하다고 제안한다.

## A. Implementation of Segmentation Proposals

- 논문 참고

# Code (Pytorch)

In [ ]:
'''FPN in PyTorch.
See the paper "Feature Pyramid Networks for Object Detection" for more details.
'''
import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.autograd import Variable

class Bottleneck(nn.Module):
    expansion = 4
    def __init__(self, in_planes, planes, stride=1):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, self.expansion*planes, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(self.expansion*planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class FPN(nn.Module):
    def __init__(self, block, num_blocks):
        super(FPN, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)

        # Bottom-up layers
        self.layer1 = self._make_layer(block,  64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)

        # Top layer
        self.toplayer = nn.Conv2d(2048, 256, kernel_size=1, stride=1, padding=0)  # Reduce channels

        # Smooth layers
        self.smooth1 = nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1)
        self.smooth2 = nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1)
        self.smooth3 = nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1)

        # Lateral layers
        self.latlayer1 = nn.Conv2d(1024, 256, kernel_size=1, stride=1, padding=0)
        self.latlayer2 = nn.Conv2d( 512, 256, kernel_size=1, stride=1, padding=0)
        self.latlayer3 = nn.Conv2d( 256, 256, kernel_size=1, stride=1, padding=0)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def _upsample_add(self, x, y):
        '''Upsample and add two feature maps.
        Args:
          x: (Variable) top feature map to be upsampled.
          y: (Variable) lateral feature map.
        Returns:
          (Variable) added feature map.
        Note in PyTorch, when input size is odd, the upsampled feature map
        with `F.upsample(..., scale_factor=2, mode='nearest')`
        maybe not equal to the lateral feature map size.
        e.g.
        original input size: [N,_,15,15] ->
        conv2d feature map size: [N,_,8,8] ->
        upsampled feature map size: [N,_,16,16]
        So we choose bilinear upsample which supports arbitrary output sizes.
        '''
        _,_,H,W = y.size()
        return F.upsample(x, size=(H,W), mode='bilinear') + y

    def forward(self, x):
        # Bottom-up
        c1 = F.relu(self.bn1(self.conv1(x)))
        c1 = F.max_pool2d(c1, kernel_size=3, stride=2, padding=1)
        c2 = self.layer1(c1)
        c3 = self.layer2(c2)
        c4 = self.layer3(c3)
        c5 = self.layer4(c4)
        # Top-down
        p5 = self.toplayer(c5)
        p4 = self._upsample_add(p5, self.latlayer1(c4))
        p4 = self.smooth1(p4)
        p3 = self._upsample_add(p4, self.latlayer2(c3))
        p3 = self.smooth2(p3)
        p2 = self._upsample_add(p3, self.latlayer3(c2))
        p2 = self.smooth3(p2)
        return p2, p3, p4, p5


def FPN101():
    # return FPN(Bottleneck, [2,4,23,3])
    return FPN(Bottleneck, [2,2,2,2])


def test():
    net = FPN101()
    fms = net(Variable(torch.randn(1,3,600,900)))
    for fm in fms:
        print(fm.size())

test()

In [ ]:
'''RetinaFPN in PyTorch.
See the paper "Focal Loss for Dense Object Detection" for more details.
'''
import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.autograd import Variable


class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, in_planes, planes, stride=1):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, self.expansion*planes, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(self.expansion*planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class RetinaFPN(nn.Module):
    def __init__(self, block, num_blocks):
        super(RetinaFPN, self).__init__()
        self.in_planes = 64

        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)

        # Bottom-up layers
        self.layer2 = self._make_layer(block,  64, num_blocks[0], stride=1)
        self.layer3 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer4 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer5 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.conv6 = nn.Conv2d(2048, 256, kernel_size=3, stride=2, padding=1)
        self.conv7 = nn.Conv2d( 256, 256, kernel_size=3, stride=2, padding=1)

        # Top layer
        self.toplayer = nn.Conv2d(2048, 256, kernel_size=1, stride=1, padding=0)  # Reduce channels

        # Smooth layers
        self.smooth1 = nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1)
        self.smooth2 = nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1)

        # Lateral layers
        self.latlayer1 = nn.Conv2d(1024, 256, kernel_size=1, stride=1, padding=0)
        self.latlayer2 = nn.Conv2d( 512, 256, kernel_size=1, stride=1, padding=0)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def _upsample_add(self, x, y):
        '''Upsample and add two feature maps.
        Args:
          x: (Variable) top feature map to be upsampled.
          y: (Variable) lateral feature map.
        Returns:
          (Variable) added feature map.
        Note in PyTorch, when input size is odd, the upsampled feature map
        with `F.upsample(..., scale_factor=2, mode='nearest')`
        maybe not equal to the lateral feature map size.
        e.g.
        original input size: [N,_,15,15] ->
        conv2d feature map size: [N,_,8,8] ->
        upsampled feature map size: [N,_,16,16]
        So we choose bilinear upsample which supports arbitrary output sizes.
        '''
        _,_,H,W = y.size()
        return F.upsample(x, size=(H,W), mode='bilinear') + y

    def forward(self, x):
        # Bottom-up
        c1 = F.relu(self.bn1(self.conv1(x)))
        c1 = F.max_pool2d(c1, kernel_size=3, stride=2, padding=1)
        c2 = self.layer2(c1)
        c3 = self.layer3(c2)
        c4 = self.layer4(c3)
        c5 = self.layer5(c4)
        p6 = self.conv6(c5)
        p7 = self.conv7(F.relu(p6))
        # Top-down
        p5 = self.toplayer(c5)
        p4 = self._upsample_add(p5, self.latlayer1(c4))
        p4 = self.smooth1(p4)
        p3 = self._upsample_add(p4, self.latlayer2(c3))
        p3 = self.smooth2(p3)
        return p3, p4, p5, p6, p7


def RetinaFPN101():
    # return RetinaFPN(Bottleneck, [2,4,23,3])
    return RetinaFPN(Bottleneck, [2,2,2,2])


def test():
    net = RetinaFPN101()
    fms = net(Variable(torch.randn(1,3,600,900)))
    for fm in fms:
        print(fm.size())

test()